In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from numpy import concatenate

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import *

import h5py

mpl.rcParams['figure.figsize'] = (10,8)
mpl.rcParams["image.origin"] = 'lower'

# hyperparameters
- encoder: 16 samples of 40 minutes, 1 interval 10 min, 1 every 4  --> 64 samples tstart
- decoder: 48 samples, 10 min == 8 hours

**Window of 114 samples**

Fighetterie per rinormalizzare input e output #fixme

In [3]:
decoder_input_and_true_v1 = np.load("decoder_input_and_true_v1.npy",mmap_mode="r")
encoder_input_v1 = np.load("encoder_input_v1.npy", mmap_mode="r")

In [4]:
decoder_input_data = decoder_input_and_true_v1[:, :, 0:-1]
decoder_target_data = decoder_input_and_true_v1[:, :, -1]
encoder_input_data = encoder_input_v1

In [5]:
decoder_target_data = np.reshape(decoder_target_data, (488520, 48,1))

In [28]:
decoder_input_data.shape

(488520, 48, 11)

In [30]:
decoder_input_data.reshape((48, 11))

ValueError: cannot reshape array of size 257938560 into shape (48,11)

In [12]:
print("decoder input shape", decoder_input_data.shape)
print("decoder target shape", decoder_target_data.shape)
print("encoder input shape", encoder_input_data.shape)

decoder input shape (488520, 48, 11)
decoder target shape (488520, 48, 1)
encoder input shape (488520, 16, 11)


In [26]:
# scalers
encoder_scaler = StandardScaler()
deconder_input_scaler = StandardScaler()
decoder_output_scaler = StandardScaler()

encoder_input_data_scaled  = encoder_scaler.fit_transform(encoder_input_data)
decoder_input_data_scaled  = decoder_input_scaler.fit_transform(decoder_input_data)
decoder_target_data_scaled = decoder_input_scaler.fit_transform(decoder_target_data)

ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [13]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, TimeDistributed, Flatten

Using TensorFlow backend.


In [22]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = len(encoder_input_v1)
encoder_input_dim = encoder_input_data.shape[2]
decoder_input_dim = decoder_input_data.shape[2]

# Define an input sequence and process it.
encoder_inputs = Input(shape=(16, encoder_input_dim))
encoder = LSTM(latent_dim, return_state=True, input_shape=(batch_size, 16, 11))
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, decoder_input_dim))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim,input_shape=(batch_size, 48, 11), return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = TimeDistributed(Dense(1, activation='sigmoid'))

decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)



In [17]:
from keras.utils import plot_model
plot_model(model, to_file='model.dot',show_shapes=True)

In [23]:
# Run training
model.compile(optimizer='adam', loss='MSE',
              metrics=['mean_squared_error'])
model.fit([encoder_input_data[:1000,], decoder_input_data[:1000,]], decoder_target_data[:1000,],
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('model_test1.h5')



Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 13s 16ms/step - loss: 0.1041 - mean_squared_error: 0.1041 - val_loss: 0.0899 - val_mean_squared_error: 0.0899
Epoch 2/100
800/800 [==============================] - 3s 4ms/step - loss: 0.0267 - mean_squared_error: 0.0267 - val_loss: 0.0288 - val_mean_squared_error: 0.0288
Epoch 3/100
800/800 [==============================] - 3s 4ms/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.0123 - val_mean_squared_error: 0.0123
Epoch 4/100
800/800 [==============================] - 3s 4ms/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0072 - val_mean_squared_error: 0.0072
Epoch 5/100
800/800 [==============================] - 4s 4ms/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0051 - val_mean_squared_error: 0.0051
Epoch 6/100
800/800 [==============================] - 3s 4ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0038 - val_m

KeyboardInterrupt: 